# 

In [127]:
import psycopg2
import pandas as pd

## Functions to interact with database

In [128]:
def create_database():
    '''Connect to default db'''
    conn = psycopg2.connect('host=localhost dbname=postgres user=postgres password=postgres')
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    # Create sparkify db with UTF8 encoding
    cur.execute('DROP DATABASE IF EXISTS accounts')
    cur.execute('CREATE DATABASE accounts')

    # Close connection to default db
    conn.close()

    # Connect to new db
    conn = psycopg2.connect('host=localhost dbname=postgres user=postgres password=postgres')
    cur = conn.cursor()

    return cur, conn

In [129]:
def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [130]:
def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

## Reading "Accounts Country" .csv file

In [131]:
dfAccountsCountry = pd.read_csv('../datasets/Wealth-AccountsCountry.csv')

In [132]:
#dfAccountsCountry.head()

In [133]:
dfAccountsCountry_clean = dfAccountsCountry[['Code', 'Short Name', 'Table Name', 'Long Name', 'Currency Unit']].dropna()

In [134]:
dfAccountsCountry_clean.head(200)

,Code,Short Name,Table Name,Long Name,Currency Unit
0,ALB,Albania,Albania,Republic of Albania,Albanian lek
1,ARG,Argentina,Argentina,Argentine Republic,Argentine peso
2,ARM,Armenia,Armenia,Republic of Armenia,Armenian dram
3,AUS,Australia,Australia,Commonwealth of Australia,Australian dollar
4,AUT,Austria,Austria,Republic of Austria,Euro
...,...,...,...,...,...
141,VNM,Vietnam,Vietnam,Socialist Republic of Vietnam,Vietnamese dong
142,PSE,West Bank and Gaza,West Bank and Gaza,West Bank and Gaza,Israeli new shekel
143,YEM,Yemen,"Yemen, Rep.",Republic of Yemen,Yemeni rial
144,ZMB,Zambia,Zambia,Republic of Zambia,New Zambian kwacha


## Reading "Accounts Data" .csv file

In [135]:
dfAccountData = pd.read_csv('../datasets/Wealth-AccountData.csv')

In [136]:
# dfAccountData.head()

In [137]:
dfAccountData.columns

Index(['Country Name', 'Country Code', 'Series Name', 'Series Code',
       '1995 [YR1995]', '1996 [YR1996]', '1997 [YR1997]', '1998 [YR1998]',
       '1999 [YR1999]', '2000 [YR2000]', '2001 [YR2001]', '2002 [YR2002]',
       '2003 [YR2003]', '2004 [YR2004]', '2005 [YR2005]', '2006 [YR2006]',
       '2007 [YR2007]', '2008 [YR2008]', '2009 [YR2009]', '2010 [YR2010]',
       '2011 [YR2011]', '2012 [YR2012]', '2013 [YR2013]', '2014 [YR2014]',
       '2015 [YR2015]', '2016 [YR2016]', '2017 [YR2017]', '2018 [YR2018]',
       'YOLO'],
      dtype='object')

In [138]:
dfAccountData = dfAccountData.drop(['YOLO'], axis=1)  # axis=1 for column; axis=0 for index (raw); default is 0.
dfAccountData = dfAccountData[['Country Name', 'Country Code', 'Series Name', 'Series Code', '1995 [YR1995]', '2000 [YR2000]', '2005 [YR2005]', '2010 [YR2010]', '2014 [YR2014]']]
dfAccountData.head()

,Country Name,Country Code,Series Name,Series Code,1995 [YR1995],2000 [YR2000],2005 [YR2005],2010 [YR2010],2014 [YR2014]
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,4.49E+10,4.36E+10,5.440000e+10,6.810000e+10,7.250000e+10
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,14072.79,14118.13,1.807230e+04,2.336286e+04,2.509002e+04
2,Albania,ALB,"Human capital per capita, employed female (con...",NW.HCA.FEMP.PC,3468.858,3424.007,2.222115e+03,2.889150e+03,2.957420e+03
3,Albania,ALB,"Human capital per capita, employed male (const...",NW.HCA.MEMP.PC,6507.931,6872.019,1.168935e+04,1.578992e+04,1.699138e+04
4,Albania,ALB,"Human capital per capita, female (constant 201...",NW.HCA.FEMA.PC,4893.006,4695.071,2.886736e+03,3.613606e+03,3.719608e+03


## Reading "Account Series" .csv file

In [139]:
dfAccountSeries = pd.read_csv('../datasets/Wealth-AccountSeries.csv')
#dfAccountSeries.head()

In [140]:
dfAccountSeries = dfAccountSeries[['Code', 'Topic', 'Indicator Name', 'Long definition']]
dfAccountSeries.head()

,Code,Topic,Indicator Name,Long definition
0,NW.HCA.TO,Human capital,Human capital (constant 2018 US$),Human capital is computed as the present value...
1,NW.HCA.PC,Human capital,Human capital per capita (constant 2018 US$),Human capital is computed as the present value...
2,NW.HCA.FEMP.PC,Human capital,"Human capital per capita, employed female (con...",Human capital is computed as the present value...
3,NW.HCA.MEMP.PC,Human capital,"Human capital per capita, employed male (const...",Human capital is computed as the present value...
4,NW.HCA.FEMA.PC,Human capital,"Human capital per capita, female (constant 201...",Human capital is computed as the present value...


## Creating tables

In [141]:
cur, conn = create_database()

In [142]:
sql_account_country_table_create = '''
    DROP TABLE IF EXISTS AccountCountry;
    CREATE TABLE AccountCountry(
        country_code VARCHAR PRIMARY KEY,
        short_name VARCHAR,
        table_name VARCHAR,
        long_name VARCHAR,
        currency_unit VARCHAR
    )
'''
cur.execute(sql_account_country_table_create)
conn.commit()

In [143]:
sql_account_data_table_create = '''
    DROP TABLE IF EXISTS AccountData;
    CREATE TABLE AccountData(
        id SERIAL PRIMARY KEY,
        country_name VARCHAR,
        country_code VARCHAR,
        series_name VARCHAR,
        series_code VARCHAR,
        year_1995 VARCHAR,
        year_2000 VARCHAR,
        year_2005 VARCHAR,
        year_2010 VARCHAR,
        year_2014 VARCHAR
    )
'''
cur.execute(sql_account_data_table_create)
conn.commit()

In [144]:
sql_account_series_table_create = '''
    DROP TABLE IF EXISTS AccountSeries;
    CREATE TABLE AccountSeries(
        id SERIAL PRIMARY KEY,
        series_code VARCHAR,
        topic VARCHAR,
        indicator_name VARCHAR,
        long_definition VARCHAR
    )
'''
cur.execute(sql_account_series_table_create)
conn.commit()

## Inserting data

In [145]:
sql_account_country_table_insert = '''
     INSERT INTO AccountCountry (
        country_code, short_name, table_name, long_name, currency_unit
    ) VALUES (
        %s, %s, %s, %s, %s
    )
'''

for i, row in dfAccountsCountry_clean.iterrows():
    cur.execute(sql_account_country_table_insert, list(row))
conn.commit()
print('"AccountCountry" data is saved')

"AccountCountry" data is saved


In [146]:
sql_account_data_table_insert = '''
     INSERT INTO AccountData (
        country_name, country_code, series_name, series_code, year_1995, year_2000, year_2005, year_2010, year_2014
    ) VALUES (
        %s, %s, %s, %s, %s, %s, %s, %s, %s
    )
'''

for i, row in dfAccountData.iterrows():
    cur.execute(sql_account_data_table_insert, list(row))
conn.commit()
print('"AccountData" data is saved')

"AccountData" data is saved


In [147]:
sql_account_series_table_insert = '''
     INSERT INTO AccountSeries (
        series_code, topic, indicator_name, long_definition
    ) VALUES (
        %s, %s, %s, %s
    )
'''

for i, row in dfAccountSeries.iterrows():
    cur.execute(sql_account_series_table_insert, list(row))
conn.commit()
print('"AccountSeries" data is saved')

"AccountSeries" data is saved
